In [12]:
import re

In [13]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
sentencesDB  = client["Sentencias"]
documentCollection = sentencesDB["Sentencias Judiciales minuscula"]
wordCollection = sentencesDB["WordTokens"]
sentenceCollection = sentencesDB["oracionTokens"]

In [14]:
#recuperar lista delitos
with open("delitos.txt", encoding="utf-8") as file:
    lista_delitos = [l.rstrip("\n") for l in file]

In [15]:
def extraerDelitosSentencia(sentencia):
    delitos = []
    for delito in lista_delitos:
        if re.search(delito, sentencia):
            delitos.append(delito)
    return delitos

In [17]:
def agregarDelitos(docs):
    for doc in docs:
        sentencia = doc['sentencia']
        query = {'_id': doc['_id']} 
        nuevo_valor = { "$set": { "delitos": extraerDelitosSentencia(sentencia)}}
        documentCollection.update_one(query,nuevo_valor)
    return docs
        
    

In [18]:
docs = documentCollection.find()
agregarDelitos(docs)

In [20]:
import pandas as pd
cursor = documentCollection.find()
df =  pd.DataFrame(list(cursor))

In [21]:
df.head()

,_id,url,sentencia,delitos
0,5e9dc2a8ae8a3bcfbdb7debf,http://tribunalsupremo.organojudicial.gob.bo/A...,\n sala \r\n plena\n \nauto \r\n supremo 12...,robo agravado
1,5e9dc2a8ae8a3bcfbdb7dec0,http://tribunalsupremo.organojudicial.gob.bo/A...,\n sala plena\n\nsentencia \r\n ...,violación
2,5e9dc2a8ae8a3bcfbdb7dec1,http://tribunalsupremo.organojudicial.gob.bo/A...,\n sala plena\n\nsentencia \r\n ...,incumplimiento de deberes
3,5e9dc2a8ae8a3bcfbdb7dec2,http://tribunalsupremo.organojudicial.gob.bo/A...,\n sala plena\n\nsentencia \r\n ...,NaN
4,5e9dc2a8ae8a3bcfbdb7dec3,http://tribunalsupremo.organojudicial.gob.bo/A...,\n sala plena\n\nsentencia \r\n ...,secuestro


In [28]:
df1 = df[df.isna().any(axis=1)]
df1.head()

,_id,url,sentencia,delitos
3,5e9dc2a8ae8a3bcfbdb7dec2,http://tribunalsupremo.organojudicial.gob.bo/A...,\n sala plena\n\nsentencia \r\n ...,NaN
6,5e9dc2a8ae8a3bcfbdb7dec5,http://tribunalsupremo.organojudicial.gob.bo/A...,\n sala plena\n\nsentencia \r\n ...,NaN
8,5e9dc2a8ae8a3bcfbdb7dec7,http://tribunalsupremo.organojudicial.gob.bo/A...,\n\n\n\nexp 9042014 contencioso administrativo...,NaN
10,5e9dc2a9ae8a3bcfbdb7dec9,http://tribunalsupremo.organojudicial.gob.bo/A...,\n\n\n\nexp 7272014 contencioso empresa uniper...,NaN
12,5e9dc2a9ae8a3bcfbdb7decb,http://tribunalsupremo.organojudicial.gob.bo/A...,\nsala plena\n\nsentencia \r\n ...,NaN


In [23]:
df1.shape


(13532, 4)

In [31]:
df1['url'][3]

'http://tribunalsupremo.organojudicial.gob.bo/AS/plena/SE-2018/se201800041.html'